In [ ]:
# define matplotlibplotting backend
# %matplotlib -l shows all available backends
%matplotlib qt

In [ ]:
import os, sys, re
sys.path.append(os.path.join(os.path.dirname(os.path.abspath(os.getcwd())), "lib/python"))

import numpy as np
from scipy.signal import butter, lfilter, freqz
from scipy import ndimage, misc

import matplotlib.pyplot as plt
import matplotlib.animation as animation

from picopic.plot_builder import PlotBuilder
from picopic.h5_reader import H5Reader

In [ ]:
##  configuration options
data_path = '/media/nfs/PiCoPiC/m50s1/'

longitude=0.035
show_grid=True
use_cache=False
autoselect = True
cmap='terrain'
image_interpolation = 'nearest'

# number of pixels in group (required for smoothing)
delta = 2

if os.path.isfile(os.path.join(data_path, "metadata.json")):
    reader = PlainReader(path = data_path, use_cache=use_cache, verbose=False)
elif os.path.isfile(os.path.join(data_path, "data.h5")):
    reader = H5Reader(path = data_path, use_cache=use_cache, verbose=False)
else:
    raise EnvironmentError("There is no corresponding data/metadata files in the path " + data_path + ". Can not continue.")

x_axis_label = r'$\mathit{t (s)}$'
y_r_axis_label = r'$\mathit{\rho_{(m^{-3})}}$'

time_range=[reader.meta.time[0], reader.meta.time[1]]
# time_range=[0, 2e-8]

size=[0, 0, reader.meta.geometry_grid[0], reader.meta.geometry_grid[1]]
r_scale = (size[2] - size[0]) / reader.meta.geometry_grid[0]
z_scale = (size[3] - size[1]) / reader.meta.geometry_grid[1]

# color limits (WARNING: clim_estimation may works incorrectly)
clim_0 = [5e16, 1.3e17]
clim_1 = [5e16, 1.3e17]

x_axis_label = r'$\mathit{Z (m)}$'
y_axis_label = r'$\mathit{R (m)}$'
cbar_axis_label = r'$V/m$'

plot_name_0 = r'$\mathbf{Electric\enspace Field\enspace Radial \enspace Component\enspace (E_r)}$'
plot_name_1 = r'$\mathbf{Electric\enspace Field\enspace Longitudial \enspace Component\enspace (E_z)}$'

In [ ]:
def butter_lowpass(cutoff, fs, order=5):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    return b, a

def butter_lowpass_filter(data, cutoff, fs, order=5):
    b, a = butter_lowpass(cutoff, fs, order=order)
    y = lfilter(b, a, data)
    return y

In [ ]:
# get data
start_frame = None # cfg.get_frame_number_by_timestamp(time_range[0])
end_frame = None # 460 # cfg.get_frame_number_by_timestamp(time_range[1])
col_number = reader.meta.get_col_by_longitude(longitude)

start_frame = end_frame = 0

for probe in reader.meta.probes:
    if (probe.shape == 'rec') and (probe.size[1] <= col_number) and (probe.size[3] >= col_number):
        start_frame = reader.meta.get_frame_number_by_timestamp(time_range[0], probe.schedule)
        end_frame = reader.meta.get_frame_number_by_timestamp(time_range[1], probe.schedule)

start_row = size[0] + delta
end_row = size[2] - delta
row_shift = 2 * delta

# filter parameters
order = 3
fs = 1e11       # sample rate, Hz
cutoff = 7e8  # desired cutoff frequency of the filter, Hz

data_rho_e = []
data_rho_i = []
data_t_e = []
data_t_i = []
data_r = []
data_z = []

for probe in reader.meta.probes:
    if (probe.shape == 'rec') and (probe.size[1] <= col_number) and (probe.size[3] >= col_number):
        if probe.component == 'temperature' and probe.specie == 'ions':
            data_t_i = reader.col_rec_range('temperature/ions', col_number, size, start_frame, end_frame)
            print("temperature ions: done")
        if probe.component == 'temperature' and probe.specie == 'electrons':
            data_t_e = reader.col_rec_range('temperature/electrons', col_number, size, start_frame, end_frame)
            print("temperature electrons: done")

        # if probe.component == 'density' and probe.specie == 'ions':
        #     data_rho_i = reader.col_rec_range('density/ions', col_number, size, start_frame, end_frame)
        #     print("density ions: done")
        # if probe.component == 'density' and probe.specie == 'electrons':
        #     data_rho_e = reader.col_rec_range('density/electrons', col_number, size, start_frame, end_frame)
        #     print("density electrons: done")
        # if probe.component == 'E/r':
        #     data_r = reader.col_rec_range('E/r', col_number, size, start_frame, end_frame)
        #     print("R: done")
        # if probe.component == 'E/z':
        #     data_z = reader.col_rec_range('E/z', col_number, size, start_frame, end_frame)
        #     print("Z: done")

# data_e_r_abs = np.abs(data_r)
# data_e_z_abs = np.abs(data_z)
# data_e_abs = np.sqrt(np.power(data_r, 2) + np.power(data_z, 2))

In [ ]:
path = '/home/cosmonaut/dev/pic/picopic_results/'
data_rho_e = np.load(path + 'data_els_0035_orig.npy')
data_rho_i = np.load(path + 'data_ions_0035_orig.npy')
data_t_e = np.load(path + 'data_t_els_0035_orig.npy')
data_t_i = np.load(path + 'data_t_ions_0035_orig.npy')
data_e_abs = np.load(path + 'data_e_0035_orig.npy')
data_e_r_abs = np.load(path + 'data_e_r_0035_orig.npy')
data_e_z_abs = np.load(path + 'data_e_z_0035_orig.npy')

# np.save('data_els_003_magn.npy', data_rho_e)
# np.save('data_ions_003_magn.npy', data_rho_i)
# np.save(path + 'data_t_els_0035_orig.npy', data_t_e)
# np.save(path + 'data_t_ions_0035_orig.npy', data_t_i)
# np.save('data_e_003_magn.npy', data_e_abs)
# np.save('data_e_r_003_magn.npy', data_e_r_abs)
# np.save('data_e_z_003_magn.npy', data_e_z_abs)

In [ ]:
# get data
start_frame = None # cfg.get_frame_number_by_timestamp(time_range[0])
end_frame = None # 460 # cfg.get_frame_number_by_timestamp(time_range[1])
col_number = reader.meta.get_col_by_longitude(longitude)

start_frame = end_frame = 0

for probe in reader.meta.probes:
    if (probe.shape == 'rec') and (probe.size[1] <= col_number) and (probe.size[3] >= col_number):
        start_frame = reader.meta.get_frame_number_by_timestamp(time_range[0], probe.schedule)
        end_frame = reader.meta.get_frame_number_by_timestamp(time_range[1], probe.schedule)

start_row = size[0] + delta
end_row = size[2] - delta
row_shift = 2 * delta

# filter parameters
order = 3
fs = 1e11       # sample rate, Hz
cutoff = 7e8  # desired cutoff frequency of the filter, Hz

data_e_f = np.zeros([end_frame - start_frame, size[2] - size[0]])
data_e_r_f = np.zeros([end_frame - start_frame, size[2] - size[0]])
data_e_z_f = np.zeros([end_frame - start_frame, size[2] - size[0]])
data_rho_e_f = np.zeros([end_frame - start_frame, size[2] - size[0]])
data_rho_i_f = np.zeros([end_frame - start_frame, size[2] - size[0]])
data_t_e_f = np.zeros([end_frame - start_frame, size[2] - size[0]])
data_t_i_f = np.zeros([end_frame - start_frame, size[2] - size[0]])

for i in range(0, len(data_e_abs[0])):
    data_e_f[:,i] = np.flip(butter_lowpass_filter(np.flip(data_e_abs[:,i]), cutoff, fs, order))
    data_e_r_f[:,i] = np.flip(butter_lowpass_filter(np.flip(data_e_r_abs[:,i]), cutoff, fs, order))
    data_e_z_f[:,i] = np.flip(butter_lowpass_filter(np.flip(data_e_z_abs[:,i]), cutoff, fs, order))
    data_rho_e_f[:,i] = np.flip(butter_lowpass_filter(np.flip(data_rho_e[:,i]), cutoff, fs, order))
    data_rho_i_f[:,i] = np.flip(butter_lowpass_filter(np.flip(data_rho_i[:,i]), cutoff, fs, order))
    data_t_e_f[:,i] = np.flip(butter_lowpass_filter(np.flip(data_t_e[:,i]), cutoff, fs, order))
    data_t_i_f[:,i] = np.flip(butter_lowpass_filter(np.flip(data_t_i[:,i]), cutoff, fs, order))

In [ ]:
data_rho_e_fm = ndimage.median_filter(data_rho_e_f, size=8)
data_rho_i_fm = ndimage.median_filter(data_rho_i_f, size=8)
data_t_e_fm = ndimage.median_filter(data_t_e_f, size=8)
data_t_i_fm = ndimage.median_filter(data_t_i_f, size=8)
data_e_fm = ndimage.median_filter(data_e_f, size=8)
data_e_r_fm = ndimage.median_filter(data_e_r_f, size=8)
data_e_z_fm = ndimage.median_filter(data_e_z_f, size=8)

In [ ]:
def latex_float(f):
    float_str = "{0:.2g}".format(f)
    if "e" in float_str:
        base, exponent = float_str.split("e")
        return r"{0:.2} \times 10^{{{1}}}".format(float(base), int(exponent))
    else:
        return float_str

In [ ]:
##### data_timeline = np.linspace(0, 0.045, 304)
data_timeline2 = np.linspace(0, 0.045, 304)

import matplotlib as mpl
# mpl.use('QT5Agg')
mpl.use('Agg')

common_line_width = 0.5

from matplotlib import rc
plt.rc('font', **{'family':'sans-serif', 'sans-serif':['DejaVu Sans'], 'size':6, 'style': 'italic'})

fig, ax = plt.subplots()
ax_e = ax.twinx()  # instantiate a second axes that shares the same x-axis

fig.set_size_inches(3.5, 1.2, True)
# fig.set_size_inches(8, 6, True)
fig.set_dpi(300)

timestamp = 7.5e-9
frmnum = reader.meta.get_frame_number_by_timestamp(timestamp, 50)

x = data_timeline
x2 = data_timeline2
line_rho_e, = ax.plot(x, data_rho_e_fm[frmnum,:304], color='black', linestyle='dotted', linewidth=common_line_width, 
                      label=r'$\mathit{\rho_{els}}$')
line_rho_i, = ax.plot(x, data_rho_i_fm[frmnum,:304], color='black', linestyle='dashed', linewidth=common_line_width,
                      label=r'$\mathit{\rho_{ions}}$')
line_e, = ax_e.plot(x2, data_e_fm[frmnum,:304], color='black', linestyle='solid', linewidth=common_line_width,
                    label=r'$\mathit{\sqrt{E_r^2 + E_z^2}}$')

fig.suptitle("t = ${}$ s".format(latex_float(timestamp)), x=.5, y=0.97)

ax.set_ylim(2e16, 8e17)
# ax.set_ylim(-2e4, 5e5)
ax_e.set_ylim(-5e5, 5e6)

ax.legend(loc='upper right', borderpad=common_line_width, fontsize='small')
# ax_e.legend(loc='best', bbox_to_anchor=(1.0, 0.55, 0.0, 0.0), borderpad=common_line_width, fontsize='small')
ax_e.legend(loc='best', bbox_to_anchor=(0.84, 1.0, 0.0, 0.0), borderpad=common_line_width, fontsize='small') # 78
ax.grid(True, linewidth=common_line_width)

for i in ax.spines.values():
    i.set_lw(common_line_width)
# ax_e.spines[:].set_lw(common_line_width)

ax.tick_params(width=common_line_width, pad=1)
ax_e.tick_params(width=common_line_width, pad=1)

ax.set_ylabel(r'$\mathit{\rho\enspace (m^{-3})}$')
# ax.set_ylabel(r'$\mathit{T\enspace (eV)}$')
ax_e.set_ylabel(r'$\mathit{E\enspace (\frac{V}{m})}$')
ax.set_xlabel(r'$\mathit{r\enspace (m)}$')

ax.yaxis.set_label_coords(-.05, .5)
ax.xaxis.set_label_coords(.5, -.2)
ax_e.yaxis.set_label_coords(1.05, .5)

ax.ticklabel_format(style='scientific', useMathText=True, scilimits=[-3,3])
ax_e.ticklabel_format(style='scientific', useMathText=True, scilimits=[-3,3])

o_left=0.075
o_right=0.920
o_top=0.860
o_bot=0.220

import matplotlib.pyplot as plt

plt.subplots_adjust(left=o_left,
                    bottom=o_bot,
                    right=o_right,
                    top=o_top,
                    wspace=0.010, 
                    hspace=0.010)

run = False

def animate(i):
    data_0_f = data_rho_e_fm[i,:304]
    data_1_f = data_rho_i_fm[i,:304]
    data_2_f = data_e_fm[i,:304]

    line_rho_e.set_ydata(data_0_f)  # update the data.
    line_rho_i.set_ydata(data_1_f)  # update the data.
    line_e.set_ydata(data_2_f)  # update the data.
    timestamp = i * reader.meta.time[2] * 50
    fig.suptitle("Time: {:.2e} s".format(timestamp), x=.85, y=.98)
    return line_rho_e, line_rho_i, line_e,

if run:
    ani = animation.FuncAnimation(
        fig, animate, interval=1, blit=True, save_count=1499)

    writer = animation.FFMpegWriter(
        fps=25, metadata=dict(artist='Me'), bitrate=1800)
    ani.save("movie_E-rho_ei_orig.mp4", writer=writer)
else:
    plt.savefig('/home/cosmonaut/dev/sci/papers/deformation-of-plasma-density-profile-under-the-action_IEEE/img/E_rho_by_r_t/E_rho_by_r_t-{}_orig.png'.format(timestamp),
                dpi=600)
# plt.show()